In [15]:

pip install lxml


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

# List of YouTube channel URLs
urls = [
    'https://www.youtube.com/channel/UC0RhatS1pyxInC00YKjjBqQ'
]


def main():
    driver = webdriver.Chrome()
    for url in urls:
        driver.get('{}/videos?view=0&sort=p&flow=grid'.format(url))
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content, 'lxml')
        titles = soup.findAll('a', id='video-title')
        views_and_time = soup.findAll('span', class_='style-scope ytd-grid-video-renderer')
        video_urls = soup.findAll('a', id='video-title')
        
        print('Channel: {}'.format(url))
        
        i = 0  # views and time
        j = 0  # urls
        total_videos = 0
        
        # Calculate the start date for the past 6 months
        today = datetime.now()
        six_months_ago = today - timedelta(days=180)
        
        for title in titles:
            video_date = views_and_time[i + 1].text
            video_date = datetime.strptime(video_date, '%b %d, %Y')
            
            if video_date >= six_months_ago:
                total_videos += 1
                print('\n{}\t{}\t{}\thttps://www.youtube.com{}'.format(title.text, views_and_time[i].text, views_and_time[i + 1].text, video_urls[j].get('href')))
            
            i += 2
            j += 1
        
        print(f'\nTotal videos in the past 6 months: {total_videos}')
        print('-' * 40)

    driver.quit()

main()


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from collections import defaultdict

# List of YouTube channel URLs
urls = [
    'https://www.youtube.com/channel/UC0RhatS1pyxInC00YKjjBqQ'
]

def main():
    driver = webdriver.Chrome()
    topic_views = defaultdict(int)
    
    for url in urls:
        driver.get('{}/videos?view=0&sort=p&flow=grid'.format(url))
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content, 'lxml')
        titles = soup.findAll('a', id='video-title')
        views_and_time = soup.findAll('span', class_='style-scope ytd-grid-video-renderer')
        
        i = 0  # views and time
        for title in titles:
            video_date = views_and_time[i + 1].text
            video_date = datetime.strptime(video_date, '%b %d, %Y')
            
            today = datetime.now()
            six_months_ago = today - timedelta(days=180)
            
            if video_date >= six_months_ago:
                views = int(views_and_time[i].text.replace(',', ''))
                topic = title.text.split(' - ')[0].strip()  # Assuming video titles are in "Topic - Title" format
                topic_views[topic] += views
            
            i += 2
    
    driver.quit()
    
    # Sort and print the most viewed topics
    sorted_topics = sorted(topic_views.items(), key=lambda x: x[1], reverse=True)
    print('Most viewed topics in the past 6 months:')
    for topic, views in sorted_topics:
        print(f'{topic}: {views} views')

main()


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from collections import defaultdict
import matplotlib.pyplot as plt

# List of YouTube channel URLs
urls = [
  'https://www.youtube.com/channel/UC0RhatS1pyxInC00YKjjBqQ'
]
]

def main():
    driver = webdriver.Chrome()
    topic_views = defaultdict(int)
    topic_lengths = defaultdict(int)
    
    for url in urls:
        driver.get('{}/videos?view=0&sort=p&flow=grid'.format(url))
        content = driver.page_source.encode('utf-8').strip()
        soup = BeautifulSoup(content, 'lxml')
        titles = soup.findAll('a', id='video-title')
        views_and_time = soup.findAll('span', class_='style-scope ytd-grid-video-renderer')
        lengths = soup.findAll('span', class_='style-scope ytd-thumbnail-overlay-time-status-renderer')
        
        i = 0  # views and time
        j = 0  # lengths
        for title in titles:
            video_date = views_and_time[i + 1].text
            video_date = datetime.strptime(video_date, '%b %d, %Y')
            
            today = datetime.now()
            six_months_ago = today - timedelta(days=180)
            
            if video_date >= six_months_ago:
                views = int(views_and_time[i].text.replace(',', ''))
                length = lengths[j].text
                topic = title.text.split(' - ')[0].strip()  # Assuming video titles are in "Topic - Title" format
                topic_views[topic] += views
                topic_lengths[topic] += int(length.split(':')[0]) * 60 + int(length.split(':')[1])
            
            i += 2
            j += 1
    
    driver.quit()
    
    # Sort topics by total video length
    sorted_topics = sorted(topic_lengths.items(), key=lambda x: x[1], reverse=True)
    
    # Plot a comparison graph
    topics = [topic for topic, _ in sorted_topics]
    views = [topic_views[topic] for topic, _ in sorted_topics]
    lengths = [topic_lengths[topic] for topic, _ in sorted_topics]
    
    fig, ax1 = plt.subplots()
    ax1.bar(topics, views, color='b', label='Views')
    ax1.set_xlabel('Topics')
    ax1.set_ylabel('Views', color='b')
    ax1.tick_params(axis='y', labelcolor='b')
    
    ax2 = ax1.twinx()
    ax2.plot(topics, lengths, color='r', marker='o', label='Video Length (seconds)')
    ax2.set_ylabel('Video Length (seconds)', color='r')
    ax2.tick_params(axis='y', labelcolor='r')
    
    plt.title('Comparison between Views and Video Length')
    plt.xticks(rotation='vertical')
    plt.legend(loc='upper left', bbox_to_anchor=(0.85, 1))
    plt.tight_layout()
    plt.show()

main()
